In [2]:
its_jupyter_notebook = True

import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
sys.path.insert(0, '..')
import ast
from tqdm.notebook import tqdm
import pickle
import seaborn as sns
from Bio import SeqIO
import re
if its_jupyter_notebook:
    sys.argv = [''] #Remove this if it's not a jupyter notebook!

In [3]:
ROOT_DIR = os.path.dirname(os.path.abspath('.'))
processed_files_dir = os.path.join(ROOT_DIR, 'dataset', 'processed_files')
rna_rna_files_dir = os.path.join(ROOT_DIR, 'dataset', 'rna_rna_pairs')
cdhit_dir = os.path.join(processed_files_dir, 'cdhit')
cl_results = os.path.join(cdhit_dir, 'cdhit_clustering.clstr') #
cl_results_fl = os.path.join(cdhit_dir, 'cdhit_clustering_fl.clstr') #
cl_results_fl2 = os.path.join(cdhit_dir, 'cdhit_clustering_fl2.clstr') #

In [4]:
df_cdna = pd.read_csv(os.path.join(processed_files_dir, "df_cdna.csv"), sep = ',')
df_genes = pd.read_csv(os.path.join(processed_files_dir, "df_genes.csv"), sep = ',')
df = pd.read_csv(os.path.join(processed_files_dir, "df_annotation_files_cleaned.csv"), sep = ',')

### Slice

In [4]:
n_clusters = 0
res = {}
fasta_sequences = SeqIO.parse(open(cl_results),'fasta')
idx = 0
with open(cl_results) as out_file:
    for _, fasta in enumerate(fasta_sequences):
        name = fasta.description 
        sequence = str(fasta.seq)
        splits = fasta.seq.split()[1:]
        cl_name = re.search('(Cluster) (\d+)', name).group(2)
        n_clusters += 1
        for sequence in splits:
            couples = [x.group(3) for x in re.finditer( r'(\d+)(\w*,>)(.*)___(.*)\.\.\..*', str(sequence))]
            genes = [x.group(4) for x in re.finditer( r'(\d+)(\w*,>)(.*)___(.*)\.\.\..*', str(sequence))]
            for i in range(len(couples)):
                couple = couples[i]
                gene = genes[i]
                res[idx] = {'couple': couple,
                          'gene':gene, 
                          'cl_name':cl_name
                         }
                idx += 1
        if _%10000 == 0:
            print(f'{_} done')
df_res = pd.DataFrame.from_dict(res, 'index').reset_index(drop = True)

df_res.to_csv(os.path.join(processed_files_dir, 'cdhit_clusters.csv'), index = False)

0 done
10000 done
20000 done
30000 done
40000 done
50000 done
60000 done
70000 done
80000 done
90000 done
100000 done
110000 done
120000 done
130000 done


In [5]:
df_res.head()

,couple,gene,cl_name
0,ENSG00000137074_ENSG00000170777,ENSG00000137074,0
1,ENSG00000137074_ENSG00000170777,ENSG00000170777,1
2,ENSG00000214031_ENSG00000248752,ENSG00000214031,2
3,ENSG00000214031_ENSG00000248752,ENSG00000248752,3
4,ENSG00000152580_ENSG00000233421,ENSG00000152580,4


### Full length 

In [15]:
n_clusters = 0
res = {}
fasta_sequences = [SeqIO.parse(open(cl_results_fl),'fasta'), SeqIO.parse(open(cl_results_fl2),'fasta')]
out_names = ['cdhit_clusters_fl', 'cdhit_clusters_fl2']
idx = 0

for k, c in enumerate([cl_results_fl, cl_results_fl2]):
    with open(c) as out_file:
        for _, fasta in enumerate(fasta_sequences[k]):
            name = fasta.description 
            splits = fasta.seq.split()[1:]
            for sequence in splits:
                genes = [x.group(3) for x in re.finditer( r'(\d+)(\w*,>)(.*)\.\.\..*', str(sequence))]
                name = re.search(r'(\d+)', name)[0]
                for i in range(len(genes)):
                    gene = genes[i]
                    res[idx] = {'gene':gene, 
                                'cl_name':name
                                }
                    idx += 1

    df_res = pd.DataFrame.from_dict(res, 'index').reset_index(drop = True)
    print(df_res.shape[0])
    print(len(df_res.cl_name.unique()))
    df_res.to_csv(os.path.join(processed_files_dir, f'{out_names[k]}.csv'), index = False)
    res = {}

41306
32456
41306
36121


In [14]:
df_res.head()

,gene,cl_name
0,ENSG00000281344,Cluster 0
1,ENSMUSG00000051747,Cluster 1
2,ENSG00000155657,Cluster 1
3,ENSG00000269821,Cluster 2
4,ENSMUSG00000101609,Cluster 3
